In [1]:
from keras.applications import VGG16
model=VGG16(weights='imagenet',
            include_top=False,
            input_shape=(224,224,3))

Using TensorFlow backend.
D:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type

In [2]:
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


In [3]:
for layer in model.layers:
    layer.trainable=False
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [4]:
from keras.models import Model
from keras.layers import Flatten
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.models import Model,Sequential
from keras.layers import Flatten,Dense
from keras.layers import Convolution2D, MaxPooling2D
img_rows=224
img_cols=224
input_shape=(img_cols, img_rows,3)
def addTopModel(bottom_model, num_classes, D=256):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = model.output
    top_model= Sequential()(top_model)
    top_model=Convolution2D(68, (3, 3), activation='relu',input_shape=input_shape)(top_model)
    top_model=MaxPooling2D( pool_size=(2, 2))(top_model)
    top_model = Flatten(name = "flatten")(top_model)
    top_model=Dense(214,activation="relu", input_shape=input_shape)(top_model)
    top_model=Dense(214,activation="relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model


In [5]:
num_classes = 3

FC_Head = addTopModel(model, num_classes)

newmodel = Model(inputs=model.input, outputs=FC_Head)

print(newmodel.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
import cv2
import numpy as np

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
def face_extractor(img):
    gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces= face_classifier.detectMultiScale(gray, 1.3,5)
    if faces is ():
        return None
    for (x,y,w,h) in faces:
        cropped_face =img[y:y+h, x:x+h]
    return cropped_face    
cap= cv2.VideoCapture("http://192.168.43.1:8080/video")
count=0
while True:
    ret,frame=cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame),(224,224))
        face= cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        file_name_path="C://Users//anika//Desktop//facerecognition//sample/train/aniket/face" + str(count)+ '.jpg'
        cv2.imwrite(file_name_path,face)
        cv2.putText(face,str(count),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
        cv2.imshow("Face Cropper",face)
                
      
    else:
        print("Face not found")
        pass
    
  
    if cv2.waitKey(1)==13 or count==400: 
        break
        
        
cap.release()
cv2.destroyAllWindows()
print("Collecting sample complete")

In [6]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'C:/Users/anika/Desktop/facerecognition/sample/train'
validation_data_dir = "C://Users//anika//Desktop//facerecognition/sample/test"

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)

train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 305 images belonging to 3 classes.
Found 105 images belonging to 3 classes.


In [7]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("face.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We have a small learning rate 
newmodel.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 305
nb_validation_samples = 105
epochs = 3
batch_size = 16

history = newmodel.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

newmodel.save("face.h5")



Epoch 1/3
19/19 [==============================] - 2687s 141s/step - loss: 0.1651 - accuracy: 0.9619 - val_loss: 3.9805e-04 - val_accuracy: 0.9167

Epoch 00001: val_loss improved from inf to 0.00040, saving model to face.h5
Epoch 2/3
19/19 [==============================] - 2229s 117s/step - loss: 0.0599 - accuracy: 0.9868 - val_loss: 0.8357 - val_accuracy: 0.9091

Epoch 00002: val_loss did not improve from 0.00040
Epoch 3/3
19/19 [==============================] - 1897s 100s/step - loss: 0.0433 - accuracy: 0.9781 - val_loss: 3.5763e-07 - val_accuracy: 1.0000

Epoch 00003: val_loss improved from 0.00040 to 0.00000, saving model to face.h5


In [8]:
from keras.models import load_model
recognizer = load_model('face.h5')

In [9]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
faces_dict = {"[0]": "Abel_Aguilar",
              "[1]": "Abel_Pacheco",
              "[2]": "aniket"
             }
def draw_text(name, pred, im):
    face = faces_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, face, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)
def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + path_class)
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    
for i in range(0,10):
    input_im = getRandomImage("C://Users//anika//Desktop//facerecognition/sample/test/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3)     
    # Get Prediction
    res = np.argmax(recognizer.predict(input_im, 1, verbose = 0), axis=1)
    # Show image with predicted class
    draw_text("Prediction", res, input_original) 
    cv2.waitKey(0)
cv2.destroyAllWindows()

Class - Abel_Pacheco
Class - Abel_Pacheco
Class - Abel_Aguilar
Class - aniket
Class - aniket
Class - aniket
Class - Abel_Aguilar
Class - aniket
Class - Abel_Aguilar
Class - aniket
